# Using the ASE interface of XTP

## Introduction

This tutorial explains how to perform calculation to predict electronic excitation using the **GWBSE** method. See [the GW Compendium: A Practical Guide to Theoretical Photoemission Spectroscopy](https://doi.org/10.3389/fchem.2019.00377), for an excellent introduction to the method.

## Requirements
* You will need to install **VOTCA** using the instructions described [here](https://github.com/votca/votca/blob/master/share/doc/INSTALL.rst)
* Once the installation is completed you need to activate the VOTCA enviroment by running the `VOTCARC.bash` script that has been installed at the bin subfolder for the path that you have provided for the installation step above
* You need to make sure that the `pyxtp` python module is accessible in your environment

## DFT + GWBSE Calculation of CH4
To run a DFT-GWBSE calculation we will use the python binding to the [xtp_tools](https://www.votca.org/xtp/xtp_tools_overview.html) calculator. We will use this calculator together with the `Atoms` class provided by ASE 

In [ ]:
from ase import Atoms
from pyxtp import xtp

The ASE interface allows to quickly create a CH4 through the library of commonly used molecule. We can then simply define and attach the `xtp` calculator to it. Many options can be passed during the creation of the `xtp` calculator. We here simply specify the number of threads that will be used by the calculator

In [ ]:
atoms = Atoms('CH4')
calc = xtp(nthreads=2)
atoms.calc = calc

## Running a calculation with the default options
To run a DFT-GWBSE calculation using the default options of XTP we can simply ask the calculator to compute for example the potential energy of the system

In [ ]:
atoms.get_potential_energy()

The previous command will run the DFT-GWBSE calculation using the aforementioned defaults. The results are store in the *Current Work Directory* in a file named `methane_summary.xml`. The results are also directly stored in the `atoms` object in a dictionnary that can be accessed via:

In [ ]:
print(atoms.results)

## Modifying the input options
The options of the calculations are specified in a data structure of the calculator called `options`. This data structure is a simple representation of the several XML files that are required to run XTP through the command line interface. You can naviguate the data strucrure with its autocomplete functionality, i.e. by pressing the `tab` key  

In [ ]:
atoms.calc.options.dftpackage.basisset

In [ ]:
atoms.calc.options.gwbse.bse.exctotal

As you can see each options also tells you what the option is for via the `_help` attribute and what is the default value via the `_default` attribute. You can set options simply via 

In [ ]:
atoms.calc.options.dftpackage.basisset = 'def2-svp' 

Frequently used options can also be set when creating the calculator. This is the case of the basis set, XC functional and charge. Hence the creating a specific `xtp` calculator via 

In [ ]:
calc = xtp(basis='def2-svp', xc='PBE', charge=1)

will create a calculator using this particular values of the basis set, functional and charge on the molecule.

We created a small options file

In [ ]:
!cat dftgwbse2.xml

In [ ]:
!xtp_tools -o dftgwbse2.xml -t 2 -e dftgwbse > dftgwbse2.log

XTP will automatically compare the default values with the user-provided and overwrites the defaults with the user input. Also, If I given property does not have a default value you can provide one using the XML file described above.

## Partial Charges
We can compute now the partial charges using the `CHELPG` method by default. For more information see the [partialcharges documentation](https://www.votca.org/xtp/partialcharges.html). Once again, we only need to provide the name of the system to compute, which in our case is `methane`.

In [ ]:
!xtp_tools -c job_name=methane -e partialcharges

## Spectrum Calculation
Finally, lets compute a convolution of the singlet spectrum using a gaussian function. For doing so, we will modify the default values for the [spectrum calculator](https://www.votca.org/xtp/spectrum.html) to compute the spectrum between 9 and 25 eV, using 1000 points in that energy range. We will use the `-c` option to modify the options accordingly. Instead we could have printed out an options file using the `xtp_tools -p spectrum` command and then modify the entries accordingly and then read them in using the `-o` option.

In [ ]:
!xtp_tools -c job_name=methane lower=9 upper=25 points=1000 -e spectrum

The results are stored in the `methane_spectrum.dat` file.

## (Optional) Plot the spectrum

We will use [matplotlib](https://matplotlib.org/), [seaborn](https://seaborn.pydata.org/) and [pandas](https://pandas.pydata.org/) libraries to plot the spectrum. You can install it using [pip](https://pip.pypa.io/en/stable/) like

In [ ]:
!pip install seaborn --user

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
columns = ["E(eV)", "epsGaussian","IM(eps)Gaussian",  "epsLorentz",  "Im(esp)Lorentz"] 
df = pd.read_table("methane_spectrum.dat", comment="#", sep='\s+',names=columns) 
sns.relplot(x="E(eV)", y="epsGaussian", ci=None, kind="line", data=df) 
plt.plot()